In [69]:
import mat73
import numpy as np
#from rigid_transform_3D import rigid_transform_3D
data_dict = mat73.loadmat('sample.mat')
#sync load file
DIC3D = data_dict['DIC3DPPresults']
#[todo] save results
#get number of images
nImages = len(DIC3D['Points3D']) # DIC3D['Points3D'] is a list
nImages

5

In [231]:
#rbm start
RBM_RotMat=[]
RBM_TransVec=[]
A = data_dict['DIC3DPPresults']['Points3D'][2][0]
B = data_dict['DIC3DPPresults']['Points3D'][0][0]
nanIndexa = np.argwhere(np.isnan(A))[:,0]
nanIndexb = np.argwhere(np.isnan(B))[:,0]
nanIndexab = np.concatenate([nanIndexa,nanIndexb])
A = np.delete(A, nanIndexab,0)
#A = np.nan_to_num(A, nan = 0)# 0 to be discussed
B = np.delete(B, nanIndexab,0)
A = A.transpose()
B = B.transpose()
A = np.asmatrix(A)
B= np.asmatrix(B)#as required by external file

In [232]:
A

matrix([[  1.08601683,   1.17790083,   1.17335975, ..., -28.00418797,
         -27.94290666, -28.14586305],
        [  0.37478153,   0.54074877,   0.55886914, ...,   6.61693949,
           6.68936462,   6.35803317],
        [253.67792973, 252.65910624, 251.59241817, ..., -14.2050448 ,
         -15.14593095, -16.24460517]])

In [233]:
#rbm
# find mean column wise different approach between matlab script
centroid_A = np.mean(A, axis=1)
centroid_B = np.mean(B, axis=1)
#approach in matlab script
#centroid_A = (1/A.shape[1])*sum(A)
#centroid_B = (1/B.shape[1])*sum(B)
print(centroid_A)
print(centroid_B)

[[  1.23175862]
 [ 26.35824221]
 [119.14285012]]
[[  0.64643916]
 [ 25.44187327]
 [119.15635719]]


In [234]:
print(centroid_A)
print(A.shape[1])
num_cols = A.shape[1]

[[  1.23175862]
 [ 26.35824221]
 [119.14285012]]
6973


In [237]:
centroid_A = centroid_A.reshape(-1, 1)
x1 = np.tile(centroid_A, (1, num_cols))
print(x1.shape,num_cols)

(3, 6973) 6973


In [238]:
print(centroid_B)
print(B.shape[0])

[[  0.64643916]
 [ 25.44187327]
 [119.15635719]]
3


In [239]:
centroid_B = centroid_B.reshape(-1, 1)
print(centroid_B)

[[  0.64643916]
 [ 25.44187327]
 [119.15635719]]


In [240]:
np.tile(centroid_B, (1, num_cols))

matrix([[  0.64643916,   0.64643916,   0.64643916, ...,   0.64643916,
           0.64643916,   0.64643916],
        [ 25.44187327,  25.44187327,  25.44187327, ...,  25.44187327,
          25.44187327,  25.44187327],
        [119.15635719, 119.15635719, 119.15635719, ..., 119.15635719,
         119.15635719, 119.15635719]])

In [241]:

# ensure centroids are 3x1 (necessary when A or B are
# numpy arrays instead of numpy matrices)
#centroid_A = centroid_A.reshape(-1, 1)
#centroid_B = centroid_B.reshape(-1, 1)

# subtract mean
Am = A - np.tile(centroid_A, (1, num_cols))
Bm = B - np.tile(centroid_B, (1, B.shape[1]))
#Am = A - centroid_A
#Bm = B - centroid_B

print(Am,Am.shape)
print(Bm,Bm.shape)

[[-1.45741793e-01 -5.38577984e-02 -5.83988785e-02 ... -2.92359466e+01
  -2.91746653e+01 -2.93776217e+01]
 [-2.59834607e+01 -2.58174934e+01 -2.57993731e+01 ... -1.97413027e+01
  -1.96688776e+01 -2.00002090e+01]
 [ 1.34535080e+02  1.33516256e+02  1.32449568e+02 ... -1.33347895e+02
  -1.34288781e+02 -1.35387455e+02]] (3, 6973)
[[   0.38962732    0.36508933    0.36457754 ...  -30.37340231
   -30.32382755  -30.23213556]
 [ -23.39251457  -23.4355393   -23.41154122 ...  -23.2071298
   -23.13023663  -22.99749412]
 [ 129.45971269  128.52499008  127.50596017 ... -128.24890738
  -129.31509013 -130.3027159 ]] (3, 6973)


In [243]:
H = np.dot(Bm,np.transpose(Am))
print(H,H.shape)

[[ 1622503.53305373  1924428.46460687  1454763.42355229]
 [ 1951331.40168605  3314188.20514582  2484208.93154535]
 [  760854.10748827   596006.44128594 39764700.45157298]] (3, 3)


In [257]:
# sanity check
#if linalg.matrix_rank(H) < 3:
#    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

# find rotation
U, S, Vt = np.linalg.svd(H)
S_mat = [[1,0,0],[0,1,0],[0,0,np.linalg.det(U*Vt)]]
#R = Vt.T * U.T * S_mat


    # special reflection case
    #if linalg.det(R) < 0:
        #print("det(R) < R, reflection detected!, correcting for it ...\n");
        #Vt[2,:] *= -1
        #R = Vt.T * U.T
print(U,'\n----------\n',Vt)

[[-0.03848744 -0.54432051 -0.83799397]
 [-0.06524979 -0.83545656  0.54566913]
 [-0.99712646  0.07568033 -0.00336221]] 
----------
 [[-0.02377656 -0.02217646 -0.9994713 ]
 [-0.54542141 -0.83756761  0.03155921]
 [-0.83782466  0.54588342  0.00781896]]


In [267]:
R = Vt.T * U.T * S_mat
#R = Vt.T * U
R = R.T# do not know why but seems that it has to be...

In [268]:
t = -R*centroid_A + centroid_B

In [269]:
print(R,"\n----\n",t)# same as data in .mat file

[[ 9.99891175e-01 -6.88270699e-04  1.47365189e-02]
 [ 5.22639257e-05  9.99070093e-01  4.31155035e-02]
 [-1.47524905e-02 -4.31100412e-02  9.98961405e-01]] 
----
 [[-2.32279468]
 [-6.02882657]
 [ 1.29172467]]


In [284]:
beforT = data_dict['DIC3DPPresults']['Points3D'][2][0]
#(beforT * R)+t
beforT*R.T

matrix([[  4.82397031,  11.31194143, 253.38228276],
        [  4.90071616,  11.43383206, 252.35600704],
        [  4.88044384,  11.40594455, 251.28971266],
        ...,
        [-28.21502756,   5.99686508, -14.06241653],
        [-28.16766815,   6.02865928, -15.00635178],
        [-28.38656505,   5.65025545, -16.0866071 ]])

In [288]:
beforT

array([[  1.08601683,   0.37478153, 253.67792973],
       [  1.17790083,   0.54074877, 252.65910624],
       [  1.17335975,   0.55886914, 251.59241817],
       ...,
       [-28.00418797,   6.61693949, -14.2050448 ],
       [-27.94290666,   6.68936462, -15.14593095],
       [-28.14586305,   6.35803317, -16.24460517]])

In [286]:
(beforT * R.T).T+t

matrix([[ 2.50117563e+00,  2.57792148e+00,  2.55764915e+00, ...,
         -3.05378222e+01, -3.04904628e+01, -3.07093597e+01],
        [ 5.28311486e+00,  5.40500549e+00,  5.37711798e+00, ...,
         -3.19614855e-02, -1.67285236e-04, -3.78571122e-01],
        [ 2.54674007e+02,  2.53647732e+02,  2.52581437e+02, ...,
         -1.27706919e+01, -1.37146271e+01, -1.47948824e+01]])

In [287]:
((beforT * R.T).T+t).T# done

matrix([[ 2.50117563e+00,  5.28311486e+00,  2.54674007e+02],
        [ 2.57792148e+00,  5.40500549e+00,  2.53647732e+02],
        [ 2.55764915e+00,  5.37711798e+00,  2.52581437e+02],
        ...,
        [-3.05378222e+01, -3.19614855e-02, -1.27706919e+01],
        [-3.04904628e+01, -1.67285236e-04, -1.37146271e+01],
        [-3.07093597e+01, -3.78571122e-01, -1.47948824e+01]])